## Linear Congruential Generator
$$
x_{t+1}=ax_t +b \pmod{m}
$$
- _Inferring a Sequence Generated by a Linear Congruence_. Joan Boyar, 1983. doi: [10.1007/978-1-4757-0602-4_32](https://doi.org/10.1007/978-1-4757-0602-4_32)
- _Cracking RNGs: Linear Congruential Generators._ Jarosław Jedynak, 2017 [tailcall.net/blog/cracking-randomness-lcgs/](https://tailcall.net/blog/cracking-randomness-lcgs/)


In [1]:
from Crypto.Util.number import getPrime, getRandomInteger, size, inverse
from Crypto.Random.random import choice
import math

In [14]:
class LCG():
    
    def __init__(self, seed: int = 0):
        primes = [getPrime(8) for i in range(4)]
        self.m = 3*choice(primes)*math.prod(primes)
        self.a = 3*choice(primes)*math.prod(primes)+1
        self.b = getRandomInteger(16)
        self.x = seed % self.m
            
    def seed(self, seed:int):
        self.x = seed % self.m
        
    def next(self):
        self.x = (self.a * self.x + self.b) % self.m
        return self.x

In [111]:
lcg = LCG(seed=getRandomInteger(4))

In [112]:
# gather initial samples, compute differences

numSamples = 10
X=[]
for i in range(numSamples):
    X.append(lcg.next())
Y = [x2 - x1 for (x2, x1) in zip(X[1:], X[:-1])]

print(X)
print(Y)

[58025290671, 81918087607, 71678390813, 27306200289, 518814161326, 406176983342, 259407311628, 78505146184, 433483132301, 184315979397]
[23892796936, -10239696794, -44372190524, 491507961037, -112637177984, -146769671714, -180902165444, 354977986117, -249167152904]


To find the modulus, compute a function $f(x)\equiv 0 \pmod{m}$, e.g.
$$
f_{i}=𝑦_{i+2}𝑦_{i}−𝑦_{i+1}𝑦_{i+1}
$$
In probability, a sequence of these values will have $gcd(f)=m$.

In [113]:
# find modulus

from functools import reduce
# reduce not necessary from python 3.9, just gcd(*f)
f=[]
for i in range(numSamples-3):
    f.append(Y[i+2]*Y[i] - Y[i+1]*Y[i+1])

print('computed modulus:')
m_hat = []
for i in range(1,len(f)):
    m_hat.append( reduce(math.gcd, f[:i]) )
    print(m_hat[-1])


computed modulus:
-1165027128228489312900
1140025290582
570012645291
570012645291
570012645291
570012645291


In [114]:
# check:
print(f'hidden modulus: {lcg.m}')
print(f'ratio: {m_hat[-1]//lcg.m}')

hidden modulus: 570012645291
ratio: 1


In [118]:
# find the multiplier, a
a_hat = []
for i in range(len(m_hat)):
    a_hat.append( Y[1]*inverse(Y[0],m_hat[i]) % m_hat[i] )
    print(a_hat[-1])

-10239696794
935230328204
467615164102
467615164102
467615164102
467615164102


In [116]:
# check:
print(f'hidden multiplier: {lcg.a}')

hidden multiplier: 467615164102


In [119]:
# finally, the increment
b_hat = []
for i in range(len(a_hat)):
    b_hat.append( X[1]-a_hat[i]*X[0] %m_hat[i] )
    print(b_hat[-1])

594161382936674896381
-81917984957
51325
51325
51325
51325


In [120]:
# check:
print(lcg.b)

51325
